(basic-inversion)=
# Estimate abundance and biomass from backscatter

In [ ]:
%run ./weight_proportions.ipynb

This method provides a means for converting $\textit{NASC}$ into number densities (animals nmi<sup>-2</sup>) via acoustic inversion by relating fish length to acoustic backscatter.

## Setting up the environment

In [10]:
from echopop.inversion import InversionLengthTS

In [11]:
# ==================================================================================================
# NASC TO POPULATION ESTIMATE CONVERSION
# ==================================================================================================
# Initialize the Inversion class
# ------------------------------
MODEL_PARAMETERS = {
    "ts_length_regression": {
        "slope": 20.,
        "intercept": -68.
    },
    "stratify_by": ["stratum_ks"],
    "expected_strata": df_dict_strata["ks"].stratum_num.unique(),
    "impute_missing_strata": True,
    "haul_replicates": True,
}

# Initiate object to perform inversion
invert_hake = InversionLengthTS(MODEL_PARAMETERS)

In [12]:
# ==================================================================================================
# Invert number density
# ---------------------

# If the above haul-averaged `sigma_bs` values were calculated, then the inversion can can 
# completed without calling in additional biodata
df_nasc_all_ages = invert_hake.invert(df_nasc=df_nasc_all_ages,
                                      df_length=[dict_df_bio["length"], dict_df_bio["specimen"]])
# ---- The average `sigma_bs` for each stratum can be inspected at:
invert_hake.sigma_bs_strata


,sigma_bs
stratum_ks,
0,0.000425
1,0.000084
2,0.000070
3,0.000114
4,0.000191
5,0.000295
6,0.000247
7,0.000384
8,0.000481


In [15]:
from echopop.nwfsc_feat import transect

# ==================================================================================================
# Set transect interval distances
# -------------------------------

# Calculate along-transect interval distances which is required for getting the area-per-interval 
# and therefore going from number density to abundance
transect.compute_interval_distance(df_nasc=df_nasc_all_ages, interval_threshold=0.05)

# ==================================================================================================
# Calculate transect interval areas
# ---------------------------------
df_nasc_all_ages["area_interval"] = (
    df_nasc_all_ages["transect_spacing"] * df_nasc_all_ages["distance_interval"]
)

In [17]:
# ==================================================================================================
# Calculate the average weights pre stratum when combining different datasets
# ---------------------------------------------------------------------------
df_averaged_weight = get_proportions.stratum_averaged_weight(
    proportions_dict=dict_df_number_proportion, 
    binned_weight_table=binned_weight_table,
    stratify_by=["stratum_ks"],
    group_by=["sex"],
)

In [18]:
# ==================================================================================================
# Calculate (and apportion) number densities to abundance, and number densities/abundance for each 
# sex 
# --------------------------------------------------------------------------------------------------

biology.compute_abundance(
    dataset=df_nasc_all_ages,
    stratify_by=["stratum_ks"],
    group_by=["sex"],
    exclude_filter={"sex": "unsexed"},
    number_proportions=dict_df_number_proportion
)

# ==================================================================================================
# Calculate (and apportion) biomass densities and biomass (from number density and abundance, 
# respectively) for the overall transect dataset as well as for each sex
# --------------------------------------------------------------------------------------------------

biology.compute_biomass(
    dataset=df_nasc_all_ages,
    stratify_by=["stratum_ks"],
    group_by=["sex"],
    df_average_weight=df_averaged_weight,
)